# VGG19 para previsão de IDC em imagens histológicas:
## Autor: Otávio A M Guerra.

## Import das bibliotecas necessárias

In [1]:
#!pip install -U tensorboardcolab
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorboardcolab import *
from keras.utils import Sequence
from google.colab import drive
drive.mount('/content/drive')
tbc=TensorBoardColab()

# Import das bibliotecas de construcao da rede neural
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from sklearn.metrics import classification_report
import matplotlib
matplotlib.use("Agg")

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
Wait for 8 seconds...
TensorBoard link:
https://a6ee28a0.ngrok.io


## Definindo função que lerá as imagens dos arquivos csv em batches para entrado na rede:

In [0]:
def csv_image_generator(inputPath, bs, mode="train", aug=None):
	# Abre o arquivo csv para leitura
	f = open(inputPath, "r")

	# Loop infinito
	while True:
		# Inicializa vetores de imagens e classes
		images = []
		labels = []

		# Loop ate atingir o BatchSize
		while len(images) < bs:
			# Lê a proxima linha do CSV
			line = f.readline()

			# Se a linha tiver vazia isso indica que chegamos ao
			# fim do arquivo
			if line == "":
				# Reseta o ponteiro do arquivo para o inicio
				# e lê novamente a primeira linha
				f.seek(0)
				line = f.readline()

				# Se estamos avaliando entao temos que sair do loop
				# para garantirmos que nao avaliaremos o modelo 2 vezes
                # em amostras do inicio do arquivo
				if mode == "eval":
					break

			# Extrai a classe da imagem e constroi a mesma
			line = line.strip().split(",")
			label = int(line[0])
			image = np.array([int(x) for x in line[1:]], dtype="uint8")
			image = image.reshape((50, 50, 3))

			# Atualiza as listas do Batch
			images.append(image)
			labels.append(label)

		# Trata parametro de Data Augmentation
		if aug is not None:
			(images, labels) = next(aug.flow(np.array(images),
				labels, batch_size=bs))

		# yield the batch to the calling function
		yield (np.array(images), labels)

## Definindo variaveis e caminhos dos arquivos:

In [0]:
# Path dos CSV's
TRAIN_CSV = "/content/drive/My Drive/TCC/Projeto/dataset/IDC_training.csv"
TEST_CSV = "/content/drive/My Drive/TCC/Projeto/dataset/IDC_test.csv"
VAL_CSV = "/content/drive/My Drive/TCC/Projeto/dataset/IDC_validation.csv"

# Numero de Epocas e Batch Size
NUM_EPOCHS = 10
BS = 128

# initialize the total number of training and testing image
NUM_TRAIN_IMAGES = 0
NUM_TEST_IMAGES = 0
NUM_VAL_IMAGES = 0

# open the training CSV file, then initialize the unique set of class
# labels in the dataset along with the testing labels
f = open(TRAIN_CSV, "r")
labels = set()
testLabels = []

# loop over all rows of the CSV file
for line in f:
	# extract the class label, update the labels list, and increment
	# the total number of training images
	label = line.strip().split(",")[0]
	labels.add(label)
	NUM_TRAIN_IMAGES += 1

# close the training CSV file and open the testing CSV file
f.close()
f = open(TEST_CSV, "r")

# loop over the lines in the testing file
for line in f:
	# extract the class label, update the test labels list, and
	# increment the total number of testing images
	label = int(line.strip().split(",")[0])
	testLabels.append(label)
	NUM_TEST_IMAGES += 1

# close the testing CSV file
f.close()

## Construindo o "gerador" de imagens:

In [0]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
	width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
	horizontal_flip=True, fill_mode="nearest")

# initialize both the training and testing image generators
trainGen = csv_image_generator(TRAIN_CSV, BS, mode="train", aug=aug)
testGen = csv_image_generator(TEST_CSV, BS, mode="train", aug=None)

## Importação e definição do modelo VGG19:

In [8]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from contextlib import redirect_stdout

img_width, img_height = 50, 50


model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

# Freeze the layers which you don't want to train.
# Here I am freezing the first 5 layers.
for layer in model.layers[:5]:
    layer.trainable = False


#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(1, activation="sigmoid")(x)

# Creating the final model 
model_final = Model(input = model.input, output = predictions)

print(model_final.summary())

# Salva o .summary() do modelo em um arquivo texto
with open('VGG19summary.txt', 'w') as f:
    with redirect_stdout(f):
        model_final.summary()

model_final.compile(loss = "binary_crossentropy", optimizer = optimizers.Adam(lr=0.0001), metrics=["accuracy"])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
W0727 18:25:15.124035 140621304436608 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0727 18:25:15.136137 140621304436608 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 50, 50, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 50, 50, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 50, 50, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 25, 25, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 25, 25, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 25, 25, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

## Definindo callbacks para salvar o modelo

In [0]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg19_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

## Treinando o modelo e realizando predições no conjunto de teste:

In [13]:
print("[START] Iniciando treinamento...\n")
H = model_final.fit_generator(
	trainGen,
	steps_per_epoch=NUM_TRAIN_IMAGES // BS,
	validation_data=testGen,
	validation_steps=NUM_TEST_IMAGES // BS,
	epochs=NUM_EPOCHS,
    callbacks = [checkpoint, early])

# re-initialize our testing data generator, this time for evaluating
testGen = csv_image_generator(TEST_CSV, BS,
	mode="eval", aug=None)

# make predictions on the testing images, finding the index of the
# label with the corresponding largest predicted probability
predIdxs = model_final.predict_generator(testGen,
	steps=(NUM_TEST_IMAGES // BS) + 1)

[START] Iniciando treinamento...

Epoch 1/10
2011/2011 [==============================] - 859s 427ms/step - loss: 0.3383 - acc: 0.8560 - val_loss: 0.3816 - val_acc: 0.8369

Epoch 00001: val_acc improved from -inf to 0.83694, saving model to vgg19_1.h5
Epoch 2/10
2011/2011 [==============================] - 842s 419ms/step - loss: 0.2939 - acc: 0.8760 - val_loss: 0.3899 - val_acc: 0.8356

Epoch 00002: val_acc did not improve from 0.83694
Epoch 3/10
2011/2011 [==============================] - 839s 417ms/step - loss: 0.2817 - acc: 0.8814 - val_loss: 0.3873 - val_acc: 0.8276

Epoch 00003: val_acc did not improve from 0.83694
Epoch 4/10
2011/2011 [==============================] - 834s 415ms/step - loss: 0.2715 - acc: 0.8861 - val_loss: 0.3530 - val_acc: 0.8518

Epoch 00004: val_acc improved from 0.83694 to 0.85176, saving model to vgg19_1.h5
Epoch 5/10
2011/2011 [==============================] - 826s 411ms/step - loss: 0.2654 - acc: 0.8891 - val_loss: 0.3315 - val_acc: 0.8633

Epoch 0000

## Plot da curva de aprendizado e Classification Report:

In [0]:
def get_label(proba):
    if proba >= 0.5:
        return 1
    else:
        return 0

predLabels = [get_label(i) for i in list(predIdxs)]


In [39]:
# show a nicely formatted classification report
print("[Métricas] Avaliando a rede no conjunto de Teste...\n")
print(classification_report(testLabels,predLabels))

[Métricas] Avaliando a rede no conjunto de Teste...

              precision    recall  f1-score   support

           0       0.86      0.90      0.88      5000
           1       0.89      0.86      0.88      5000

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



In [40]:
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
print("Acuracia Balanceada: {}".format(balanced_accuracy_score(testLabels,predLabels)))
print("ROC-AUC: {}".format(roc_auc_score(testLabels,predLabels)))

Acuracia Balanceada: 0.8778999999999999
ROC-AUC: 0.8778999999999999


In [0]:
# plot the training loss and accuracy
N = 10
plt.style.use("ggplot")
plt.figure(figsize=(15,10))
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="test_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="test_acc")
plt.title("Loss e Acurácia no Conjunto de Treino e Teste")
plt.xlabel("Epoca #")
plt.ylabel("Loss/Acurácia")
plt.legend(loc="lower left")
plt.savefig("plot.png")